In [1]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import cv2, time
import numpy as np
import argparse, pprint
from matplotlib import pyplot as plt
import open3d as o3
from open3d import JVisualizer
# from pyntcloud import PyntCloud

import pyrealsense2 as rs

from scripts.devices.d415_camera import CameraD415
from scripts.VBOATS import VBOATS

# from scripts.devices.r200_camera import CameraR200

from scripts.hyutils.sys_utils import *
from scripts.hyutils.img_utils import *
from scripts.hyutils.debug_utils import *
from scripts.hyutils.plotting_utils import *
from scripts.hyutils.uv_mapping_utils import *

pp = pprint.PrettyPrinter(indent=4)

vboat = VBOATS()
vboat.debug = False
vboat.dead_x = 0
vboat.dead_y = 3

In [2]:
def create_disparity_img(_depth, focal, baseline, dscale=0.001):
    tmp2 = _depth*dscale
    loc = np.where(tmp2 == 0.0)
    tmp2[loc] = 1.0
    disparity = (focal*baseline)/tmp2
    disparity[loc] = 0.0
    dmax = np.max(disparity)
    disparity2uintGain = (255)/(dmax)
    print(dmax, disparity2uintGain)
    disparity = np.uint8(disparity*disparity2uintGain)
    return disparity, disparity2uintGain

In [ ]:
ctx = rs.context()
for d in ctx.devices:
    print ('Found device: ', \
    d.get_info(rs.camera_info.name), ' ', \
    d.get_info(rs.camera_info.serial_number))
dev = ctx.devices[0]


In [ ]:
cam.close()

In [ ]:
cam.__del__()

In [3]:
cam = CameraD415(flag_save=False,use_statistics=False,fps=30, verbose=True)

('Found device: ', u'Intel RealSense D415', ' ', u'847412062231')
[INFO] CameraD415() -- Resetting Hardware...
[INFO] CameraD415() -- Initialization successful!!!


In [4]:
extr = cam.get_extrinsics()
intr = cam.get_intrinsics()

pp.pprint(extr)
pp.pprint(intr)

rotation: [0.999999, 0.000126028, 0.00157422, -0.000125856, 1, -0.000109517, -0.00157424, 0.000109319, 0.999999]
translation: [0.0147318, -6.67465e-05, -0.00013899]
{   u'color': width: 640, height: 480, ppx: 322.508, ppy: 234.413, fx: 614.156, fy: 613.426, model: 2, coeffs: [0, 0, 0, 0, 0],
    u'depth': width: 640, height: 480, ppx: 319.742, ppy: 242.007, fx: 596.392, fy: 596.392, model: 4, coeffs: [0, 0, 0, 0, 0]}


In [ ]:
pts_path = "1.ply"
fragment = o3.io.read_point_cloud(pts_path)
visualizer = o3.JVisualizer()
visualizer.add_geometry(fragment)
visualizer.show()

In [8]:
# pc = rs.pointcloud()
points = cam.get_pointcloud()
# arra = np.asarray(points.get_vertices())

In [21]:
tmp = np.asarray(points.get_vertices())
# print(tmp.shape,tmp)
print(tmp[0])

(-0., -0., 0.)


In [22]:
tmp = np.concatenate([points]).T
print(tmp.shape)

(288582, 3)


In [ ]:
import rospy
from std_msgs.msg import Header
from sensor_msgs import point_cloud2
from sensor_msgs.msg import Image,CameraInfo,CompressedImage, PointCloud2, PointField

pc2_msg_fields = [PointField('x', 0, PointField.FLOAT32, 1),PointField('y', 4, PointField.FLOAT32, 1),PointField('z', 8, PointField.FLOAT32, 1)]

header = Header()
header.seq = 0
header.stamp = rospy.Time.now()
header.frame_id = "d415/depth_optical_frame"
# pc2_msg = point_cloud2.create_cloud(header, self.pc2_msg_fields, points )

In [23]:
print(points[0,:])

TypeError: list indices must be integers, not tuple

In [ ]:
pc.map_to(cam.frames.get_color_frame())
points = pc.calculate(cam.frames.get_depth_frame())
# vtx = np.asarray(points.get_vertices())
# points.("1.ply", cam.frames.get_color_frame())

In [5]:
rgb, depth = cam.read()
disparity, _ = create_disparity_img(depth,cam.intrinsics["depth"].fx,cam.extrinsics.translation[0], cam.dscale)
print(np.max(disparity))
pplot(depth,"depth")
pplot(disparity,"disparity")
pplot(rgb,"rgb")

(6.508079, 39.182068620195246)
255


In [ ]:
raw_umap, raw_vmap, dt = vboat.get_uv_map(disparity,timing=True)
pplot(raw_umap,"raw_umap")
pplot(raw_vmap,"raw_vmap")

In [ ]:
normvmap = cv2.normalize(raw_vmap,None,0.0, 1.0, cv2.NORM_MINMAX)
equmap = cv2.equalizeHist(raw_vmap)
pplot(normvmap,"norm vmap")
pplot(equmap,"equmap vmap")

In [ ]:
_,thresh = cv2.threshold(raw_vmap, 8, 255,cv2.THRESH_TOZERO)
pplot(thresh,"thresh")
# print(np.argmax(disparity))

In [ ]:
rgba, deptha = cam.read(flag_aligned=True)
pplot(depth,"depth aligned")
pplot(rgb,"rgb aligned")

In [ ]:
t0 = tmp.timestamp

In [ ]:
frames = cam.pipeline.wait_for_frames()
tmp = frames.get_depth_frame()
print(tmp.width, tmp.height)
print(tmp.timestamp - t0)

In [ ]:
# ======================================================

In [ ]:
boo

In [ ]:
cam = CameraR200()

In [ ]:
intr,scale = cam.get_intrinsics()
extr, b = cam.get_extrinsics()

In [ ]:
rgb, depth = cam.read()
pplot(depth,"hey")
pplot(rgb,"rgb")

In [ ]:
tmp = depth/avgDisparity
tmp = np.uint8(tmp*255)
tmp = cv2.cvtColor(tmp,cv2.COLOR_GRAY2BGR)
pplot(tmp,"normalized depth")

In [ ]:
cam.close()

In [ ]:
# ======================================================

In [ ]:
rgb, depth = cam.read()
grey = cv2.cvtColor(rgb,cv2.COLOR_BGR2GRAY)

pplot(depth,"First Raw Depth")
pplot(rgb,"First Raw RGB")
pplot(grey,"First Greyscale RGB")

In [ ]:
rgb2, depth2 = cam.read()
grey2 = cv2.cvtColor(rgb2,cv2.COLOR_BGR2GRAY)

pplot(depth2,"Second Raw Depth")
pplot(rgb2,"Second Raw RGB")
pplot(grey2,"Second Greyscale RGB")

In [ ]:
# sift = cv2.xfeatures2d.SIFT_create()
# surf = cv2.xfeatures2d.SURF_create()
orb = cv2.ORB_create(nfeatures=3000)

In [ ]:
# keypoints_sift, descriptors = sift.detectAndCompute(grey, None)
# keypoints_surf, descriptors = surf.detectAndCompute(grey, None)
kp1, desc1 = orb.detectAndCompute(grey, None)
kp2, desc2 = orb.detectAndCompute(grey2, None)

In [ ]:
# siftimg = cv2.drawKeypoints(grey, keypoints_sift, None)
# pplot(siftimg,"Sift Features")

# surfimg = cv2.drawKeypoints(grey, keypoints_surf, None)
# pplot(surfimg,"Surf Features")

orbimg1 = cv2.drawKeypoints(grey, kp1, None)
pplot(orbimg1,"ORB Features 1")
orbimg2 = cv2.drawKeypoints(grey2, kp2, None)
pplot(orbimg2,"ORB Features 2")

In [ ]:
print(256/65536.0)